In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

In [ ]:
to_generate_data = False
to_train = False
to_test = True

*Data Preparation*

In [2]:
if to_generate_data:
    df = pd.read_parquet('autodl-tmp/mins_data.parquet')
    df.sort_values(by=['START_TIME', 'Attraction'], inplace=True)
    df['lag1'] = df.groupby('Attraction')['WAIT_TIME_MAX'].shift(1).fillna(0)
    df['lag2'] = df.groupby('Attraction')['WAIT_TIME_MAX'].shift(2).fillna(0)
    df.reset_index(inplace = True,drop = True)
    one_hot_columns = [col for col in df.columns if 
                       pd.api.types.is_numeric_dtype(df[col]) and 
                       sorted(df[col].unique()) == [0, 1]]
    non_one_hot_columns = [col for col in df.columns if 
                           pd.api.types.is_numeric_dtype(df[col]) and 
                           col not in one_hot_columns and col not in ['lag1','lag2','WAIT_TIME_MAX']]
    scaler = StandardScaler()
    df[non_one_hot_columns] = scaler.fit_transform(df[non_one_hot_columns])
    
    num_attractions = len(df.Attraction.unique())
    grouped = df.groupby('START_TIME')
    feature_columns = [i for i in list(df) if i not in ['WORK_DATE',
     'START_TIME',
     'Attraction',
     'WAIT_TIME_MAX',
     'DEB_TIME',
     'FIN_TIME',
     'DEB_TIME_x',
     'FIN_TIME_x',
     'DEB_TIME_y',
     'FIN_TIME_y']]
    zero_data = np.zeros((1, len(feature_columns)))
    zero_df = pd.DataFrame(zero_data, columns=feature_columns)
    processed_groups = []
    targets = []
    all_attractions = sorted(df['Attraction'].unique())
    for time_point, group in tqdm(grouped):
        processed_group = group.copy()
        processed_group = processed_group.drop_duplicates(subset=['Attraction'], keep='first')
        present_attractions = group['Attraction'].unique()
        missing_attractions = set(all_attractions) - set(present_attractions)
        for attraction in missing_attractions:
            temp_df = zero_df.copy()
            temp_df['Attraction'] = attraction
            temp_df['START_TIME'] = time_point
            processed_group = pd.concat([processed_group, temp_df], ignore_index=True)
        processed_group.sort_values(by=['Attraction'], inplace=True)
        t = processed_group.pop('WAIT_TIME_MAX').fillna(0)
        data = torch.tensor(processed_group[feature_columns].values, dtype=torch.float)
        processed_groups.append(data)
        targets.append(t)
        if data.shape[0] != 26:
            break
    features_tensor = torch.stack(processed_groups)
    labels_tensor = torch.tensor(np.array(targets))
    total_samples = len(features_tensor)
    # last 10% for validation
    split_idx = int(total_samples * 0.9)
    
    train_features = features_tensor[:split_idx]
    train_labels = labels_tensor[:split_idx]
    validation_features = features_tensor[split_idx:]
    validation_labels = labels_tensor[split_idx:]
    
    train_dataset = TensorDataset(train_features, train_labels)
    validation_dataset = TensorDataset(validation_features, validation_labels)
    torch.save(train_dataset, 'autodl-tmp/train_dataset.pth')
    torch.save(validation_dataset, 'autodl-tmp/validation_dataset.pth')
else:
    train_dataset = torch.load('autodl-tmp/train_dataset.pth')
    validation_dataset = torch.load('autodl-tmp/validation_dataset.pth')
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=1, shuffle=False)

100%|██████████| 40600/40600 [06:50<00:00, 98.89it/s] 


*Network design*

In [3]:
class Network1(nn.Module):
    def __init__(self, input_size, output_size, num_layers=1):
        super(Network1, self).__init__()
        self.fc1 = nn.Linear(input_size, 1500)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(1500, 1000)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(1000, output_size)
        self.relu = nn.LeakyReLU()
        init.kaiming_normal_(self.fc1.weight, nonlinearity='leaky_relu')
        init.kaiming_normal_(self.fc2.weight, nonlinearity='leaky_relu')
        init.kaiming_normal_(self.fc3.weight, nonlinearity='leaky_relu')
    def forward(self, x, return_intermediate=False):
        intermediate_outputs = {}
        out = self.relu(self.fc1(x))
        intermediate_outputs['fc1'] = out
        out = self.dropout1(out)
        intermediate_outputs['dropout1'] = out
        out = self.relu(self.fc2(out))
        intermediate_outputs['fc2'] = out
        out = self.dropout2(out)
        intermediate_outputs['dropout2'] = out
        out = self.fc3(out)
        intermediate_outputs['fc3'] = out
        
        if return_intermediate:
            return intermediate_outputs
        else:
            return out

class Network2(nn.Module):
    def __init__(self,input_size,output_size):
        super(Network2, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 256)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(256,128)
        self.dropout3 = nn.Dropout(0.5)
        self.fc4 = nn.Linear(128, output_size)
        self.relu = nn.LeakyReLU()
        init.kaiming_normal_(self.fc1.weight, nonlinearity='leaky_relu')
        init.kaiming_normal_(self.fc2.weight, nonlinearity='leaky_relu')
        init.kaiming_normal_(self.fc3.weight, nonlinearity='leaky_relu')
        init.kaiming_normal_(self.fc4.weight, nonlinearity='leaky_relu')
    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.dropout1(out)
        out = self.relu(self.fc2(out))
        out = self.dropout2(out)
        out = self.relu(self.fc3(out))
        out = self.dropout3(out)
        out = self.fc4(out)
        return out

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
network1 = Network1(input_size=num_attractions*len(feature_columns),
                    output_size=num_attractions*num_attractions).to(device)
network2 = Network2(input_size=num_attractions, output_size=num_attractions).to(device)


In [4]:
def build_initial_matrix(out):
    matrix = out.view(26, 26)
    normalized_matrix = matrix / matrix.sum(dim=1, keepdim=True)
    return normalized_matrix
def calculate_stationary_matrix(matrix, num_iterations=30, epsilon=1e-6):
    v = torch.rand(matrix.size(0), dtype=matrix.dtype, device=matrix.device)
    v = v / v.sum() 
    for _ in range(num_iterations):
        v_next = torch.mv(matrix, v)
        v_next = v_next / v_next.sum()  
        if torch.norm(v - v_next) < epsilon:
            break
        v = v_next
    return v
def Middle(out):
    return calculate_stationary_matrix(build_initial_matrix(out))

In [5]:
if to_train:
    criterion = nn.MSELoss()
    optimizer1 = optim.Adam(network1.parameters(), lr=0.001)
    optimizer2 = optim.Adam(network2.parameters(), lr=0.001)

*Evaluation Design*

In [11]:
def validate(model1, model2, validation_loader, criterion, device):
    model1.eval() 
    model2.eval()
    val_loss = 0.0
    with torch.no_grad(): 
        for inputs, targets in validation_loader:
            inputs = inputs.reshape((1,num_attractions*len(feature_columns))).float()
            inputs, targets = inputs.to(device), targets.float().to(device)
            out1 = model1(inputs)
            intermediate_matrix = build_initial_matrix(out1)
            stationary_matrix = calculate_stationary_matrix(intermediate_matrix)
            predictions = model2(stationary_matrix)
            loss = criterion(predictions, targets)
            val_loss += loss.item()
    return np.sqrt(val_loss / len(validation_loader))
def RMSE_for_Attraction(network1,network2):
    network1.eval() 
    network2.eval()
    attraction = processed_group.Attraction
    result = pd.DataFrame({'attraction':attraction,'RMSE':[0 for i in attraction]})
    val_loss = 0.0
    with torch.no_grad(): 
        for inputs, targets in tqdm(validation_loader):
            targets = targets.reshape(26)
            inputs = inputs.reshape((1,num_attractions*len(feature_columns))).float()
            inputs, targets = inputs.to(device), targets.float().to(device)
            out1 = network1(inputs)
            intermediate_matrix = build_initial_matrix(out1)
            stationary_matrix = calculate_stationary_matrix(intermediate_matrix)
            predictions = network2(stationary_matrix)
            for i in range(len(predictions)):
                index = result['attraction'] == attraction[i]
                result.loc[index, 'RMSE'] += float((targets[i] - predictions[i]))**2
    result.RMSE = result.RMSE/len(validation_loader)
    result.RMSE = result.RMSE.apply(lambda x:np.sqrt(x))
    return result

*Model Training*

In [7]:
if to_train:
    num_epochs = 10
    current_best = 1e10
    RMSE_val = []
    for epoch in range(num_epochs):
        network1.train()
        network2.train()
        train_loss = 0.0
        pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}")
        for batch_idx, (inputs, targets) in pbar:
            inputs = inputs.reshape((1,num_attractions*len(feature_columns))).float()
            inputs, targets = inputs.to(device), targets.float().to(device)
            optimizer1.zero_grad()
            optimizer2.zero_grad()
            out1 = network1(inputs)
            stationary_matrix = Middle(out1)
            predictions = network2(stationary_matrix).reshape((1,26)).float()
            loss = criterion(predictions, targets)
            loss.backward()
            clip_value = 1 #1-10
            nn.utils.clip_grad_norm_(network1.parameters(), clip_value) 
            nn.utils.clip_grad_norm_(network2.parameters(), clip_value)
            optimizer1.step()
            optimizer2.step()
            train_loss += loss.item()
            pbar.set_postfix({'Train Loss': train_loss / (batch_idx + 1)})
            
        val_loss = validate(network1, network2, validation_loader, criterion, device)
        RMSE_val.append(val_loss)
        if np.sqrt(val_loss) < current_best:
            torch.save(network1, 'network1.pth')
            torch.save(network2, 'network2.pth')
            current_best = val_loss
        print(f"Validation Loss: {val_loss:.4f}")

Epoch 1: 100%|██████████| 36540/36540 [11:05<00:00, 54.93it/s, Train Loss=362] 


Validation Loss: 33.4984


Epoch 2: 100%|██████████| 36540/36540 [12:26<00:00, 48.95it/s, Train Loss=1.91e+4]


Validation Loss: 21.0608


Epoch 3: 100%|██████████| 36540/36540 [13:51<00:00, 43.94it/s, Train Loss=1.18e+4]


...

Validation Loss: 15.2540


Epoch 9: 100%|██████████| 36540/36540 [18:57<00:00, 32.11it/s, Train Loss=1.52e+5]


Validation Loss: 16.5614


Epoch 10:   4%|▍         | 1380/36540 [00:43<17:42, 33.09it/s, Train Loss=1.78e+5]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Epoch 10: 100%|██████████| 36540/36540 [18:45<00:00, 32.46it/s, Train Loss=2e+4]   


Validation Loss: 24.9686


*Best Model Evaluation*

In [13]:
if to_test:
    best_network1 = torch.load('autodl-tmp/network1.pth')
    best_network2 = torch.load('autodl-tmp/network2.pth')
    val_loss = validate(network1, network2, validation_loader, criterion, device)
    print('RMSE for all Attractions',val_loss)
    RMSE_for_Attraction(best_network1,best_network2)

RMSE for all Attractions 25.199518133114832


100%|██████████| 4060/4060 [02:15<00:00, 29.88it/s]


,attraction,RMSE
0,Bumper Cars,5.671665
1,Bungee Jump,14.057108
2,Circus Train,3.391123
3,Crazy Dance,4.080013
4,Dizzy Dropper,8.861856
5,Drop Tower,18.153843
6,Flying Coaster,10.705520
7,Free Fall,24.476853
8,Giant Wheel,16.283435
9,Giga Coaster,12.038203
